In [18]:
import numpy as np
import matplotlib.pyplot as plt


def grad_desc_with_const_lr(grad, lr, epoch, x):
    points = np.zeros((epoch, 2))
    points[0] = x

    for i in range(1, epoch):
        x = x - lr * np.array(grad(x[0], x[1]))
        points[i] = x

    return points


[[2.00000000e+01 2.00000000e+01]
 [1.64000000e+01 1.64000000e+01]
 [1.34480000e+01 1.34480000e+01]
 [1.10273600e+01 1.10273600e+01]
 [9.04243520e+00 9.04243520e+00]
 [7.41479686e+00 7.41479686e+00]
 [6.08013343e+00 6.08013343e+00]
 [4.98570941e+00 4.98570941e+00]
 [4.08828172e+00 4.08828172e+00]
 [3.35239101e+00 3.35239101e+00]
 [2.74896063e+00 2.74896063e+00]
 [2.25414771e+00 2.25414771e+00]
 [1.84840113e+00 1.84840113e+00]
 [1.51568892e+00 1.51568892e+00]
 [1.24286492e+00 1.24286492e+00]
 [1.01914923e+00 1.01914923e+00]
 [8.35702370e-01 8.35702370e-01]
 [6.85275943e-01 6.85275943e-01]
 [5.61926274e-01 5.61926274e-01]
 [4.60779544e-01 4.60779544e-01]
 [3.77839226e-01 3.77839226e-01]
 [3.09828166e-01 3.09828166e-01]
 [2.54059096e-01 2.54059096e-01]
 [2.08328459e-01 2.08328459e-01]
 [1.70829336e-01 1.70829336e-01]
 [1.40080056e-01 1.40080056e-01]
 [1.14865646e-01 1.14865646e-01]
 [9.41898293e-02 9.41898293e-02]
 [7.72356601e-02 7.72356601e-02]
 [6.33332413e-02 6.33332413e-02]
 [5.193325